In [ ]:
# Script name       : Nv_viruses_alignment_summary_script
# Written by        : Yael Hazan
# Date              : 15-July-2018
# script description: This script creates a table that summarize the alignments of trinity sequences (contigs)
#                     against viruses, Nv and bacteria sequences. 
#             Step 1: Load the blast output files into df's. 
#             Step 2: Create a table (df) that summarize trinity sequences alignments.
#             Step 3: Writing the table to a csv file.
#             Step 4: Create a fasta file with virus only sequences. 

In [ ]:
DIR = r'/cs/labs/michall/yaelh/Projects/Nematostella_viruses_project/Processed_files'
       

import os
import numpy as np
import pandas as pd 
from Bio import SeqIO

In [ ]:
# Step 1: Load the blast input files into df's (blast of trinity sequences against viruses, Nv and bacteria):

columns = ['query_id', 'subject_id', 'pct_identity', 'aln_length', 'n_of_mismatches', \
           'gap_openings', 'q_start', 'q_end', 's_start', 's_end', 'e_value', 'bit_score']

trin_virus_df = pd.read_table(os.path.join(DIR, 'Trinity_virus_e10_tabular_txid.out'), header = None, names=columns)
trin_Nv_df = pd.read_table(os.path.join(DIR, 'Trinity_NV_e5_tabular.out'), header = None, names=columns)
trin_bac_df = pd.read_table(os.path.join(DIR, 'Trinity_bacteriaarchea_e10_tabular.out'), header = None, names=columns)

# reset the index for the trinity-virus df (the only one that contains description):
trin_virus_df = trin_virus_df.reset_index()

# rename columns:
trin_virus_df = trin_virus_df.rename(columns = {'index':'Contig', 'subject_id':'Virus_accession', \
                                                'e_value':'Virus_E_value', 'q_start':'Virus_q_start', 'q_end':'Virus_q_end'})
trin_Nv_df = trin_Nv_df.rename(columns = {'query_id':'Contig', 'e_value':'Nv_E_value', \
                                          'q_start':'Nv_q_start', 'q_end':'Nv_q_end'})
trin_bac_df = trin_bac_df.rename(columns = {'query_id':'Contig', 'e_value':'Bacteria_E_value', \
                                            'q_start':'Bacteria_q_start', 'q_end':'Bacteria_q_end'})

# drop unnecessary columns:
trin_virus_df = trin_virus_df[['Contig', 'Virus_accession', 'Virus_E_value', 'Virus_q_start', 'Virus_q_end']]
trin_Nv_df = trin_Nv_df[['Contig', 'Nv_E_value', 'Nv_q_start', 'Nv_q_end']]
trin_bac_df = trin_bac_df[['Contig', 'Bacteria_E_value', 'Bacteria_q_start', 'Bacteria_q_end']]


In [ ]:
# Step 2: Create a table (df) that summarize trinity sequnces alignments:

merge_virus_Nv = pd.merge(trin_virus_df, trin_Nv_df, on =['Contig'], how = 'outer')
trin_align_summary_df = pd.merge(merge_virus_Nv, trin_bac_df, on =['Contig'], how = 'outer')





In [ ]:
# Step 3: Writing the table to a csv file:

trin_align_summary_df.to_csv(os.path.join(DIR,'trin_align_summary.csv.gz'),index = False, compression = 'gzip')

In [ ]:
# (use this if I want to not display the NaN values)
trin_align_summary_df.fillna('').head()

In [ ]:
trin_align_summary_df.head()

In [ ]:
# Step 4: Create a fasta file with virus only sequences. Here I extract from the trinity fasta file sequences 
#         that were aligned only to virus database (and not to Nematostella or bacteria). In addition, only the part 
#         of the sequence that was aligned will be extracted and written to a new fasta file, not the whole original 
#         sequence (only from q_start to q_end).
 

virus_only_contigs = []
mini_temp_list = []
trinity_fasta_dic = {}

# Create a list of lists with virus only contigs, q_start and q_end :
for i in range(len(trin_align_summary_df)):
    if  pd.isna(trin_align_summary_df['Nv_E_value'][i]) \
    and pd.isna(trin_align_summary_df['Bacteria_E_value'][i]):
        mini_temp_list = [trin_align_summary_df['Contig'][i], \
                          trin_align_summary_df['Virus_q_start'][i], \
                          trin_align_summary_df['Virus_q_end'][i]]
        
        virus_only_contigs.append(mini_temp_list)
        

        
# Load the trinity fasta file into a dictionary:
for record in SeqIO.parse(os.path.join(DIR, 'Trinity.fasta'), 'fasta'):
        trinity_fasta_dic[record.id] = str(record.seq)
        

# Create the new fasta file with virus only sequences:
with open(os.path.join(DIR, 'Trinity_virus.fasta'), "w") as f:
    for i in range(len(virus_only_contigs)):
        contig = virus_only_contigs[i][0]

#       When start > end make a correction (start and end will switch:

        if int(virus_only_contigs[i][1]) > int(virus_only_contigs[i][2]):
            start = int(virus_only_contigs[i][2]) - 1
            end = int(virus_only_contigs[i][1])
            
#       Otherwise, no need for correction:            
        else:
            start = int(virus_only_contigs[i][1]) - 1
            end = int(virus_only_contigs[i][2]) 
                    
        seq = trinity_fasta_dic[contig][start:end]
            
            
        f.write('>' + contig + '\n' + seq + '\n')